In [48]:
#setup
import json
import sqlite3
import re

In [49]:
# initiate sql tables
conn = sqlite3.connect('gsi.sqlite')
cur = conn.cursor()

cur.executescript('''
DROP TABLE IF EXISTS Projects;
DROP TABLE IF EXISTS Retrofits;
DROP TABLE IF EXISTS Joint;
DROP TABLE IF EXISTS Grants;
CREATE TABLE Projects (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    pname TEXT UNIQUE,
    grant_id INTEGER,
    grant_amount INTEGER,
    approve_yr INTEGER
);
CREATE TABLE Retrofits (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    rname TEXT UNIQUE
);
CREATE TABLE Grants (
    id INTEGER NOT NULL PRIMARY KEY UNIQUE,
    gname TEXT UNIQUE
);
CREATE TABLE Joint (
    number INTEGER,
    project_id INTEGER,
    retro_id INTEGER,
    UNIQUE (project_id, retro_id)
)''')

grants = [(0,'SMIP'),(1,'GARP')]
cur.executemany('INSERT INTO Grants (id, gname) VALUES (?, ?)', grants)

fname = input('Enter file name: ')
if len(fname) < 1 : fname = 'GSI_Private_Projects_Retrofit.geojson'

with open(fname) as f :
    dict_data = json.loads(f.read())


for f in dict_data['features'] :
     project = f['properties']

     #if project['NAME'] != project['PROJECTNAME'] : print(project['NAME'], project['PROJECTNAME'])
     if project['APPROVALDATE'] is None : continue

     #handle missing name fields
     if project['NAME'] is not None : pname = project['NAME']
     elif project['PROJECTNAME'] is not None : pname = project['PROJECTNAME']
     else :
         print('Error in Record: ', project['TRACKINGNUMBER'])
         continue
     grant_amount = project['GRANTAMOUNT']
     #parse approval year
     approve_date = project['APPROVALDATE']
     approve_yr = int(approve_date[:4])
     #get retrofit names from json keys
     kl = list(project.keys())
     rname = kl[11:]
     # get grant values
     if project['SMIP'] == -1 : grant_id = 0
     elif project['GARP'] == -1 : grant_id = 1
     else : grant_id = None

     for i in range(len(rname)) :
          rname[i] = (rname[i],) # make rname into a list of tuples for executemany argument

     cur.execute('''INSERT OR IGNORE INTO Projects (pname, grant_id, grant_amount, approve_yr)
     VALUES (?, ?, ?, ?)''', (pname, grant_id, grant_amount, approve_yr))
     cur.execute('SELECT id FROM Projects WHERE pname = ?', (pname,))
     project_id = cur.fetchone()[0]

     stmt = 'INSERT OR IGNORE INTO Retrofits (rname) VALUES (?)'
     cur.executemany(stmt, rname)

     rstr = kl[11:] #get retrofit names as list of strings to loop through
     for i in range(len(rstr)) :
         if project[rstr[i]] > 0 :
             number = project[rstr[i]]
             cur.execute('SELECT id FROM Retrofits WHERE rname = ?', rname[i])
             retro_id = cur.fetchone()[0]

             #create many-to-many join table
             cur.execute('''INSERT OR REPLACE INTO Joint (number, project_id, retro_id)
             VALUES (?, ?, ?)''', (number, project_id, retro_id))
         else : continue

conn.commit()


Enter file name: 


In [50]:
sqlstmt = '''SELECT Grants.gname, SUM(Projects.grant_amount), COUNT(Projects.pname)
FROM Grants JOIN Projects ON Grants.id = Projects.grant_id
GROUP BY Grants.gname'''

for row in cur.execute(sqlstmt) :
    print('For %s grant: %d dollars allocated to %d projects' % (row[0], row[1], row[2]))

For GARP grant: 39753820 dollars allocated to 57 projects
For SMIP grant: 28774888 dollars allocated to 74 projects


In [51]:
#Same same but different: this is the notation I tend to run into (tables as A,B)
# in an interview they're likely to use it as it tests familiarity. ugh.
sqlstmt = '''SELECT A.gname, SUM(B.grant_amount), COUNT(B.pname)
FROM Grants A JOIN Projects B ON A.id = B.grant_id
GROUP BY A.gname'''

for row in cur.execute(sqlstmt) :
    print('For %s grant: %d dollars allocated to %d projects' % (row[0], row[1], row[2]))

For GARP grant: 39753820 dollars allocated to 57 projects
For SMIP grant: 28774888 dollars allocated to 74 projects


In [14]:
import pandas as pd

In [16]:
#I want to SEE the data, et an idea of structure, etc
query = 'SELECT * FROM GRANTS'
df = pd.read_sql_query(query, conn)
df.head(10)
# so only 2 rows, 2 cols

id gname
0   0  SMIP
1   1  GARP

In [57]:
#I want to SEE the data, et an idea of structure, etc
query = 'SELECT * FROM PROJECTS'
df = pd.read_sql_query(query, conn)
print(df.head(5))
# many rols, cols
print(df.describe())
len(df) # 155 rows

   id                        pname  grant_id  grant_amount  approve_yr
0   1             Weavers Way Coop       NaN           NaN        2011
1   2               CATCH Retrofit       NaN           NaN        2010
2   3     Pennypack Woods Retrofit       NaN           NaN        2010
3   4     Community Legal Services       NaN           NaN        2013
4   5  TVPV Stormwater Credit App.       NaN           NaN        2011
               id    grant_id  grant_amount   approve_yr
count  155.000000  131.000000  1.180000e+02   155.000000
mean    79.051613    0.435115  5.807518e+05  2015.806452
std     45.472525    0.497675  7.484805e+05     2.279487
min      1.000000    0.000000  1.565900e+04  2010.000000
25%     40.500000    0.000000  1.168845e+05  2015.000000
50%     79.000000    0.000000  3.537940e+05  2016.000000
75%    117.500000    1.000000  7.693705e+05  2018.000000
max    158.000000    1.000000  5.677050e+06  2020.000000


155

In [64]:
#I want to look at grants with amounts in them
df.dropna(thresh = 2) # drop rows with 2 NA values (id, amount)
# that didn't pick up 'NaN's...

# this method is better anyway, allows me to specify the column
df[df['grant_amount']>=8000]
# len(df[df['grant_amount']>=8000])

id                                              pname  grant_id  \
13    14  Pennypack Woods Homeowners Association Stormwater       0.0   
16    17                           13410- 13420 Damar Drive       0.0   
17    18         GSFS, Green Street Friends School Retrofit       0.0   
18    19                                1148 Wharton Street       0.0   
19    20                        Methodist Home Rain Gardens       0.0   
..   ...                                                ...       ...   
147  150  PEER GARP/OVERBROOK PRESBYTERIAN CHURCH STORMW...       1.0   
148  152                       2001 W Lehigh Ave - Site 240       1.0   
149  153                                 2230 Castor Avenue       1.0   
152  156               Eastern State Penitentiary - Phase I       0.0   
154  158            First Presbyterian Church in Germantown       0.0   

     grant_amount  approve_yr  
13       135500.0        2013  
16       255000.0        2014  
17        91080.0        2013  
18        79500.0        2014  
19        70000.0        2013  
..            ...         ...  
147      475525.0        2019  
148     1123200.0        2019  
149     1555000.0        2019  
152      117750.0        2019  
154      240000.0        2020  

[118 rows x 5 columns]

# How much money was given out by each grant program?

In [47]:
#How much money was given out by each grant program?
sqlstmt = '''SELECT Grants.gname, SUM(Projects.grant_amount), COUNT(Projects.pname)
FROM Grants JOIN Projects ON Grants.id = Projects.grant_id
GROUP BY Grants.gname'''

for row in cur.execute(sqlstmt) :
    print('For %s grant: %d dollars allocated to %d projects' % (row[0], row[1], row[2]))


For GARP grant: 39753820 dollars allocated to 57 projects
For SMIP grant: 28774888 dollars allocated to 74 projects


In [103]:
# set df to projects
query = 'SELECT * FROM PROJECTS'
proj_df = pd.read_sql_query(query, conn)

In [104]:
projCount = proj_df.groupby('grant_id')[['pname']].count()
projCount

pname
grant_id       
0.0          74
1.0          57

In [105]:
grantSum = proj_df.groupby('grant_id')[['grant_amount']].sum()
grantSum

grant_amount
grant_id              
0.0         28774888.0
1.0         39753820.0

In [98]:
df2 = projCount
# print(df2)
# df2[['grant_amount']] = grantSum[['grant_amount']]
df2 = df2.join(grantSum, on = 'grant_id')
df2

pname  grant_amount
grant_id                     
0.0          74    28774888.0
1.0          57    39753820.0

In [102]:
query = 'SELECT * FROM GRANTS'
grants_df = pd.read_sql_query(query, conn)


summary_df = pd.merge(df2, grants_df, left_on='grant_id', right_on='id')
summary_df

pname  grant_amount  id gname
0     74    28774888.0   0  SMIP
1     57    39753820.0   1  GARP

# How many of each type of retrofit was done per grant?

In [ ]:
#I also want this to display the totals for each retrofit for a "Null" grant group...
sqlstmt = '''SELECT coalesce(Grants.gname, 'No grant'), Retrofits.rname, SUM(Joint.number)
FROM Projects LEFT JOIN Grants LEFT JOIN Retrofits LEFT JOIN Joint
ON Joint.project_id = Projects.id AND Joint.retro_id = Retrofits.id
AND Projects.grant_id = Grants.id
GROUP BY 2, 1'''

# for row in cur.execute(sqlstmt) :
#     print(row[0], row[1], row[2])


In [108]:
query = 'SELECT * FROM Joint'
joint_df = pd.read_sql_query(query, conn)
joint_df.head()

number  project_id  retro_id
0       1           1         2
1       2           2         2
2       1           3         5
3       1           4         8
4       2           5         3

In [122]:
query = 'SELECT * FROM Retrofits'
retro_df = pd.read_sql_query(query, conn)
retro_df.head()

id                          rname
0   1     SURFACE_INFILTRATION_BASIN
1   2  SUBSURFACE_INFILTRATION_BASIN
2   3        SURFACE_DETENTION_BASIN
3   4     SUBSURFACE_DETENTION_BASIN
4   5                BIOINFILTRATION

In [126]:
df2 = pd.merge(joint_df, proj_df, left_on = 'project_id', right_on = 'id')
df2.head()

number  project_id  retro_id  id                        pname  grant_id  \
0       1           1         2   1             Weavers Way Coop       NaN   
1       2           2         2   2               CATCH Retrofit       NaN   
2       1           3         5   3     Pennypack Woods Retrofit       NaN   
3       1           4         8   4     Community Legal Services       NaN   
4       2           5         3   5  TVPV Stormwater Credit App.       NaN   

   grant_amount  approve_yr  
0           NaN        2011  
1           NaN        2010  
2           NaN        2010  
3           NaN        2013  
4           NaN        2011

In [134]:
retro = pd.merge(df2.groupby(['grant_id', 'retro_id'])[['number']].sum().reset_index(), retro_df,
                left_on = 'retro_id', right_on = 'id')
retro.head()

grant_id  retro_id  number  id                          rname
0       0.0         1       8   1     SURFACE_INFILTRATION_BASIN
1       1.0         1       1   1     SURFACE_INFILTRATION_BASIN
2       0.0         2      30   2  SUBSURFACE_INFILTRATION_BASIN
3       1.0         2      43   2  SUBSURFACE_INFILTRATION_BASIN
4       0.0         3      16   3        SURFACE_DETENTION_BASIN

In [135]:
summary_df = pd.merge(retro, grants_df,
                      left_on = 'grant_id', right_on = 'id')
summary_df

grant_id  retro_id  number  id_x                          rname  id_y  \
0        0.0         1       8     1     SURFACE_INFILTRATION_BASIN     0   
1        0.0         2      30     2  SUBSURFACE_INFILTRATION_BASIN     0   
2        0.0         3      16     3        SURFACE_DETENTION_BASIN     0   
3        0.0         4      35     4     SUBSURFACE_DETENTION_BASIN     0   
4        0.0         5      33     5                BIOINFILTRATION     0   
5        0.0         6      19     6                   BIORETENTION     0   
6        0.0         8       4     8                     GREEN_ROOF     0   
7        0.0         9       3     9                POROUS_PAVEMENT     0   
8        0.0        10       2    10            WQ_TREATMENT_DEVICE     0   
9        1.0         1       1     1     SURFACE_INFILTRATION_BASIN     1   
10       1.0         2      43     2  SUBSURFACE_INFILTRATION_BASIN     1   
11       1.0         3       6     3        SURFACE_DETENTION_BASIN     1   
12       1.0         4      90     4     SUBSURFACE_DETENTION_BASIN     1   
13       1.0         5       1     5                BIOINFILTRATION     1   
14       1.0         6       3     6                   BIORETENTION     1   

   gname  
0   SMIP  
1   SMIP  
2   SMIP  
3   SMIP  
4   SMIP  
5   SMIP  
6   SMIP  
7   SMIP  
8   SMIP  
9   GARP  
10  GARP  
11  GARP  
12  GARP  
13  GARP  
14  GARP